In [ ]:
import mysql.connector

# il blocco try/except che fornisce un modello per qualsiasi operazione relativa al database da eseguire nello script
try:
    cnx = mysql.connector.connect(user = 'root', password='passw',       #connessione al database
                                  host = '127.0.0.1',
                                  database = 'sampledb')
    cursor = cnx.cursor() #L’oggetto cursor fornisce i mezzi per l’esecuzione delle istruzioni e l’interfaccia per il recupero dei risultati
    # definire le righe dei dipendenti
    emps = [
        (9001, "Jeff Russel", "sales"),
        (9002, "Jane Boorman", "sales"),
        (9003, "Tom Heints", "sales")
    ]
    # definire la query
    query_add_emp = ("""INSERT INTO emps (empno, empname, job) VALUES (%s, %s, %s)""")
    # inserire le righe dei dipendenti
    for emp in emps:
        cursor.execute(query_add_emp, emp)    #si inseriscono le riga una alla volta

    # definire e inserire gli stipendi
    salary = [
        (9001, 3000),
        (9002, 2800),
        (9003, 2500)
    ]
    query_add_salary = ("""INSERT INTO salary (empno, salary) VALUES (%s, %s)""")

    for sal in salary:
        cursor.execute(query_add_salary, sal)

    # definire e inserire gli ordini
    orders = [
        (2608, 9001, 35),
        (2617, 9001 ,35),
        (2620, 9001, 139),
        (2621, 9002, 95),
        (2626, 9002, 218)
    ]
    query_add_order = ("""INSERT INTO orders(pono, empno, total) VALUES (%s, %s, %s)""")
    for order in orders:
        cursor.execute(query_add_order, order)

    # rendere permanenti gli inserimenti nel database
    cnx.commit()

except mysql.connector.Error as err:
    print("Error-Code:", err.errno)
    print("Error-Message: {}".format(err.msg))#La clausola finally viene eseguita in ogni caso . In questa clausola, si chiude esplicitamente il cursore e quindi la connessione
finally:
    cursor.close()
    cnx.close()

Error-Code: 1062
Error-Message: Duplicate entry '9001' for key 'emps.PRIMARY'


In [ ]:
import mysql.connector
#interrogare il database
try:
    cnx = mysql.connector.connect(user='root', password='passw',
    host='127.0.0.1',
    database='sampledb')
    cursor = cnx.cursor()
    query = ("SELECT * FROM emps WHERE empno > %s")
    empno = 9001
    cursor.execute(query, (empno,))
    for (empno, empname, job) in cursor:
        print("{}, {}, {}".format(
        empno, empname, job))
except mysql.connector.Error as err:
    print("Error-Code:", err.errno)
    print("Error-Message: {}".format(err.msg))
#La clausola finally viene eseguita in ogni caso . In questa clausola, si chiude esplicitamente il cursore e quindi la connessione
finally:
    cursor.close()
    cnx.close()


try:
    cnx = mysql.connector.connect(user = 'root', password = 'passw',
                                  host='127.0.0.1',
                                  database = 'sampledb')
    cursor = cnx.cursor()
    query = ("""SELECT e.empno, e.empname, e.job, s.salary 
                FROM emps e JOIN salary s ON e.empno = s.empno
                WHERE e.empno > %s""")
    empno = 9001
    cursor.execute(query, (empno,))
    for (empno, empname, job, salary) in cursor:
        print("{}, {}, {}, {}".format(empno, empname, job, salary))
except mysql.connector.Error as err:
    print("Error-Code:", err.errno)
    print("Error-Message: {}".format(err.msg))
#La clausola finally viene eseguita in ogni caso . In questa clausola, si chiude esplicitamente il cursore e quindi la connessione
finally:
    cursor.close()
    cnx.close()

9002, Jane Boorman, sales
9003, Tom Heints, sales
9002, Jane Boorman, sales, 2800
9003, Tom Heints, sales, 2500


In [ ]:
#pagina 15
import yfinance as yf

data = []
tickers = ['TSLA','ORCL','AMZN']
for ticker in tickers:
    tkr = yf.Ticker(ticker)
    hist = tkr.history(period = '5d').reset_index()
    records = hist[['Date','Close']].to_records(index = False) 
    records = list(records)
    records = [(ticker, str(elem[0])[:10], round(elem[1])) for elem in records]
    data = data + records
print(data)

#per salvare questi dati nella tabelal stocks
import mysql.connector
try:
    cnx = mysql.connector.connect(user = 'root', password = 'passw',
                                  host='127.0.0.1',
                                  database = 'sampledb')
    cursor = cnx.cursor()
    query_add_stocks = ("""INSERT INTO stocks(ticker, date, price) VALUES (%s, %s, %s)""")
    #aggiungere righe del prezzo delle azioni
    cursor.executemany(query_add_stocks, data)
    cnx.commit()
except mysql.connector.Error as err:
    print("Error-Code:", err.errno)
    print("Error-Message: {}".format(err.msg))
#La clausola finally viene eseguita in ogni caso . In questa clausola, si chiude esplicitamente il cursore e quindi la connessione
finally:
    cursor.close()
    cnx.close()

#per verificare l'aggiunta eseguire questi comandi su terminale mysql
'''SELECT
date,
ticker,
price,
LAG(price) OVER(PARTITION BY ticker ORDER BY date) AS prev_price #LAG() nella lista SELECT è una funzione SQL analitica. Consente di accedere ai dati di una riga precedente dalla riga corrente
FROM stocks;'''

#le righe dei soli titoli i cui prezzi non sono scesi di oltre l’1% rispetto al prezzo del giorno precedente nel periodo specificato
'''
SELECT s.* FROM stocks AS s
LEFT JOIN
(SELECT DISTINCT(ticker) FROM
(SELECT price/LAG(price) OVER(PARTITION BY ticker ORDER BY date) AS dif, ticker FROM stocks) AS b
WHERE dif <0.99) AS a
ON a.ticker = s.ticker
WHERE a.ticker IS NULL;
'''
# pag 22
import pandas as pd
from mysql.connector import errorcode
try:
    cnx = mysql.connector.connect(user = 'root', password = 'passw',
                                  host='127.0.0.1',
                                  database = 'sampledb')
    query = ('''SELECT s.* FROM stocks as s
                LEFT JOIN (SELECT DISTINCT(ticker) FROM (SELECT price/LAG(price) OVER(PARTITION BY ticker ORDER BY date) AS dif, ticker FROM stocks) AS b WHERE dif < 0.99) as a ON a.ticker = s.ticker WHERE a.ticker IS NULL''')
    df_stocks = pd.read_sql(query, con = cnx)
    df_stocks = df_stocks.set_index(['ticker','date'])
except mysql.connector.Error as err:
    print("Error-Code:", err.errno)
    print("Error-Message: {}".format(err.msg))
finally:
    cnx.close()
print(df_stocks)

[('TSLA', '2025-05-29', 358), ('TSLA', '2025-05-30', 346), ('TSLA', '2025-06-02', 343), ('TSLA', '2025-06-03', 344), ('TSLA', '2025-06-04', 332), ('ORCL', '2025-05-29', 163), ('ORCL', '2025-05-30', 166), ('ORCL', '2025-06-02', 167), ('ORCL', '2025-06-03', 169), ('ORCL', '2025-06-04', 170), ('AMZN', '2025-05-29', 206), ('AMZN', '2025-05-30', 205), ('AMZN', '2025-06-02', 207), ('AMZN', '2025-06-03', 206), ('AMZN', '2025-06-04', 206)]
                   price
ticker date             
ORCL   2025-05-29  163.0
       2025-05-30  166.0
       2025-06-02  167.0
       2025-06-03  169.0
       2025-06-04  170.0
AMZN   2025-05-29  206.0
       2025-05-30  205.0
       2025-06-02  207.0
       2025-06-03  206.0
       2025-06-04  207.0
ORCL   2025-05-29  163.0
       2025-05-30  166.0
       2025-06-02  167.0
       2025-06-03  169.0
       2025-06-04  170.0
AMZN   2025-05-29  206.0
       2025-05-30  205.0
       2025-06-02  207.0
       2025-06-03  206.0
       2025-06-04  207.0
ORCL   2025-05

C:\Users\lorenzo.pourpour\AppData\Local\Temp\ipykernel_8364\1374245901.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stocks = pd.read_sql(query, con = cnx)
